In [133]:
import numpy as np
import pandas as pd

threshold = 0.5

In [2]:
train_embedding = np.load('./data/BERT_encoding/train_encode_tuned.npy')

In [144]:
dev_embedding = np.load('./data/BERT_encoding/dev_encode_tuned.npy')
dev_embedding = dev_embedding[0: 300]

In [145]:
train_embedding.shape

(291862, 768)

In [146]:
dev_embedding.shape

(300, 768)

In [143]:
dev_data = pd.read_csv('./data/dev_set_v1.txt', sep='\t', header=None)
dev_data = dev_data[0: 300]

In [7]:
train_data = pd.read_csv('./data/train_set_v1.txt', sep='\t', header=None)

### Load keras model

In [8]:

import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.optimizers import Adam

seed = 7
np.random.seed(seed)

model = Sequential()
model.add(Dense(units=768, activation='relu', input_dim=1536))
# model.add(Dense(units=50, activation='relu', input_dim=x_train.shape[1]))
model.add(Dropout(0.3))
model.add(Dense(units=2, activation='softmax'))
# optimizer = Adam(lr=0.01)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam', metrics=['accuracy'])

model.summary()
# SVG(model_to_dot(model).create(prog='dot', format='svg'))

# callbacks
filepath="best_weights_tough_head.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
earlyStopping = EarlyStopping(monitor='val_acc', patience=1, verbose=0, mode='auto')

callbacks_list = [checkpoint, earlyStopping]

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 768)               1180416   
_________________________________________________________________
dropout_1 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1538      
Total params: 1,181,954
Trainable params: 1,181,954
Non-trainable params: 0
_________________________________________________________________


In [128]:
model.load_weights("pair_classifier_weights.hdf5")
# y_test = model.predict(x_dev, batch_size=128, verbose=1)


In [152]:
%%time
top_k = 10

pairs = np.concatenate((np.repeat([dev_embedding[0]], [len(train_embedding)], axis=0), train_embedding), axis=1)
result = model.predict(pairs, batch_size=1024, verbose=1)

291862/291862 [==============================] - 1s 3us/step
CPU times: user 1.27 s, sys: 748 ms, total: 2.02 s
Wall time: 1.9 s


In [157]:
top_k_idx=result[:,1].argsort()[::-1][0:top_k]

In [161]:
np.array(train_data[0])[top_k_idx]

array([3220, 7907, 2717,  176, 1698, 3225, 5523, 5523, 3220, 1017])

In [162]:
counts = np.bincount(np.array(train_data[0])[top_k_idx])
np.argmax(counts)

3220

In [ ]:
correct = 0

for i in range(len(dev_data)):
    pairs = np.concatenate((np.repeat([dev_embedding[i]], [len(train_embedding)], axis=0), train_embedding), axis=1)
    result = model.predict(pairs, batch_size=1024, verbose=1)
#     max_index = np.argmax(result[:,1])
    top_k_idx=result[:,1].argsort()[::-1][0:top_k]
    np.array(train_data[0])[top_k_idx]
    counts = np.bincount(np.array(train_data[0])[top_k_idx])
    
    if dev_data[0][i] == np.argmax(counts):
        print("correct")
        correct += 1

correct

291862/291862 [==============================] - 1s 3us/step


In [134]:
result = model.predict(pairs, batch_size=1024, verbose=1)

291862/291862 [==============================] - 1s 3us/step


In [135]:
result[result[:,1] > threshold].shape

(24746, 2)

In [139]:
max_index = np.argmax(result[:,1])
#np.array(train_data[0])[result[:,1] > threshold]
print(result[max_index])
train_data[0][max_index]

[0.00761543 0.99238455]


7565

In [141]:
dev_data[0][3]

3088